In [1]:
import numpy as np
import adapt
from adapt.feature_based import DANN
from tensorflow import keras
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical 
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


def get_encoder(input_shape=(512,)):
    model = Sequential()
    model.add(Dense(512, activation='relu',
                    input_shape=input_shape))
    model.add(Dense(256, activation="relu"))
    model.add(Dense(128, activation="sigmoid"))
    return model

def get_task(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(34, activation="softmax"))
    return model

def get_discriminator(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

save_check_pt = './checkpoints_DANN'
# Sample to load model
#Dummy inputs to load model
xs=np.zeros((8446, 512))
ys=np.zeros((8446, 34))
xt=np.zeros((762, 512))
yt=np.zeros((762, 34))
weather_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
weather_model.fit(xs, ys,xt,yt, epochs=0,verbose=1,batch_size = 32) # NO fit happen, just dummy step to load model
weather_model.load_weights(save_check_pt+ '/weather')

#sample input and predict
sample_in = np.zeros(512) #Simulates flatten 512 pixels from 32 * 16 orginal image
predction = weather_model.predict(sample_in.reshape([-1,512]))
print("sample one picture predcict shape with one hot encoding",np.shape(predction))# prediction is one-hot encoding
print("one picture actual outcome",np.argmax(predction)) # actual predict outcome

#sample input with two pictures and predction
sample_in = np.zeros((3,512))
predction = weather_model.predict(sample_in)
print("sample one picture predcict shape with one hot encoding",np.shape(predction))# prediction is one-hot encoding
print("one picture actual outcome",np.argmax(predction,axis=1)) # actual predict outcome

sample one picture predcict shape with one hot encoding (1, 34)
one picture actual outcome 25
sample one picture predcict shape with one hot encoding (3, 34)
one picture actual outcome [25 25 25]
